In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os

import json
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/ustc/Argus/timeout60'
df = pd.read_csv(out_dir+"/USTC-TFC16_argus_60.csv")
df.head()

,SrcId,Rank,StartTime,LastTime,Trans,Flgs,Seq,Dur,RunTime,IdleTime,Mean,StdDev,Sum,Min,Max,SrcMac,DstMac,SrcOui,DstOui,SrcAddr,DstAddr,Proto,Sport,Dport,sTos,dTos,sDSb,dDSb,sCo,dCo,sTtl,dTtl,sHops,dHops,sIpId,dIpId,sMpls,dMpls,AutoId,sAS,dAS,iAS,Cause,NStrok,sNStrok,dNStrok,TotPkts,SrcPkts,DstPkts,TotBytes,SrcBytes,DstBytes,TotAppByte,SAppBytes,DAppBytes,PCRatio,Load,SrcLoad,DstLoad,Loss,SrcLoss,DstLoss,pLoss,Retrans,SrcRetra,DstRetra,pRetran,SrcGap,DstGap,Rate,SrcRate,DstRate,Dir,SIntPkt,SIntDist,SIntPktAct,SIntActDist,SIntPktIdl,SIntIdlDist,DIntPkt,DIntDist,DIntPktAct,DIntActDist,DIntPktIdl,DIntIdlDist,SrcJitter,SrcJitAct,DstJitter,DstJitAct,State,Label,srcUdata,dstUdata,SrcWin,DstWin,sVlan,dVlan,sVid,dVid,sVpri,dVpri,SRange,ERange,SrcTCPBase,DstTCPBase,TcpRtt,SynAck,AckDat,TcpOpt,label
0,0.0.0.0,NaN,18:06:12.700750,18:06:12.699986,NaN,,0,0.000764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,man,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,Start,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,normal
1,0.0.0.0,1.0,02:52:17.371558,02:52:17.371767,1.0,e,1,0.000209,0.000209,1.722964e+09,0.000209,0.0,0.000209,0.000209,0.000209,NaN,NaN,NaN,NaN,1.1.242.235,1.2.222.94,tcp,28466,https,0.0,0.0,cs0,cs0,NaN,NaN,32.0,32.0,0.0,0.0,0x501d,0xb01d,NaN,NaN,0,NaN,NaN,NaN,Start,NaN,NaN,NaN,2,1,1,1545,70,1475,0,0,0,-0.0,0.0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,4784.688965,0.0,0.0,<?>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON,NaN,NaN,NaN,65535.0,23168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.035113e+09,1.829255e+09,0.0,0.0,0.0,T,normal
2,0.0.0.0,2.0,02:52:17.371559,02:52:17.371605,1.0,e,2,0.000046,0.000046,1.722964e+09,0.000046,0.0,0.000046,0.000046,0.000046,NaN,NaN,NaN,NaN,1.1.221.81,1.2.185.2,tcp,4540,https,0.0,0.0,cs0,cs0,NaN,NaN,32.0,32.0,0.0,0.0,0x4fa7,0x43a4,NaN,NaN,0,NaN,NaN,NaN,Start,NaN,NaN,NaN,2,1,1,1545,70,1475,0,0,0,-0.0,0.0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,21739.130859,0.0,0.0,<?>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON,NaN,NaN,NaN,65535.0,23168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.833866e+09,1.454900e+09,0.0,0.0,0.0,T,normal
3,0.0.0.0,3.0,02:52:17.371560,02:52:17.371609,1.0,e,3,0.000049,0.000049,1.722964e+09,0.000049,0.0,0.000049,0.000049,0.000049,NaN,NaN,NaN,NaN,1.1.88.78,1.2.207.187,tcp,52340,https,0.0,0.0,cs0,cs0,NaN,NaN,32.0,32.0,0.0,0.0,0x511d,0xb31d,NaN,NaN,0,NaN,NaN,NaN,Start,NaN,NaN,NaN,2,1,1,1545,70,1475,0,0,0,-0.0,0.0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,20408.164062,0.0,0.0,<?>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON,NaN,NaN,NaN,65535.0,23168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.011655e+09,1.484449e+09,0.0,0.0,0.0,T,normal
4,0.0.0.0,4.0,02:52:17.371562,02:52:17.371562,1.0,e,4,0.000000,0.000000,1.722964e+09,0.000000,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,1.1.155.98,1.2.226.139,tcp,59203,https,0.0,NaN,cs0,NaN,NaN,NaN,32.0,NaN,0.0,NaN,0xd0c2,NaN,NaN,NaN,0,NaN,NaN,NaN,Start,NaN,NaN,NaN,1,1,0,70,70,0,0,0,0,-0.0,0.0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.000000,0.0,0.0,?>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.956989e+09,NaN,0.0,0.0,0.0,T,normal


In [2]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/RF/RF_ustc_argus_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

In [3]:
# Features from paper 1 and paper 2
cols = ["Dur", "RunTime", "IdleTime", "Mean", "StdDev", "Sum", "Min", "Max", "Proto", "Cause", "TotPkts", "SrcPkts", "DstPkts", "TotBytes", "SrcBytes",
         "DstBytes", "Load", "SrcLoad", "DstLoad", "Rate", "SrcRate", "DstRate", "label",
        "SAppBytes", "DAppBytes", "SynAck", "AckDat", "TcpRtt"
        ]

len(cols)

28

In [4]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

# Training

In [5]:
best_f1 = 0
worst_f1 = 1
best_mean, worst_mean, best_std, worst_std = None, None, None, None

save=True

for timeout in timeouts:
    print("Processing timeout : ", timeout)
    
    if timeout == 'default':
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/ustc/Argus/{timeout}/USTC-TFC16_argus_{timeout}.csv'
    else:
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/ustc/Argus/timeout{timeout}/USTC-TFC16_argus_{timeout}.csv'
    
    
    df = pd.read_csv(out_dir)
    df = df[cols]
    
    #df = df.drop(columns=['SrcId', 'SrcAddr', 'DstAddr', 'Sport', 'Dport', 'StartTime', 'LastTime', 'Dir', 'State', 'sIpId', 'sDSb', 'AutoId'])
    df = df.dropna() 
    # Convert categorical variables to numerical using Label Encoding
    label_encoders = {}
    for column in ['Proto', 'Cause', 'Flgs' ]:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['label'])  # Assuming 'label' is the target variable
    y = df['label']
    
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        #report = classification_report(y_test, pred,  target_names=le.classes_, digits=4)
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')

        
    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.8275742769991676, 'accMean': 0.8359566475768212, 'recMean': 0.8131556284299961, 'precMean': 0.8670304581699891}
Std of all scores:  {'f1Std': 0.0149376432703667, 'accStd': 0.01410905418100656, 'recStd': 0.012861033404104414, 'precStd': 0.021556175425099612}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.8275742769991676, 'accMean': 0.8359566475768212, 'recMean': 0.8131556284299961, 'precMean': 0.8670304581699891}
Std of all scores:  {'f1Std': 0.0149376432703667, 'accStd': 0.01410905418100656, 'recStd': 0.012861033404104414, 'precStd': 0.021556175425099612}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all sco

In [6]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  2
Mean Scores (Best):  {'f1Mean': 0.835244284839866, 'accMean': 0.8512225767112994, 'recMean': 0.8224650269329421, 'precMean': 0.8674685219946902}
Std Scores (Best): {'f1Std': 0.023729731748337944, 'accStd': 0.027006963879671667, 'recStd': 0.02435031226220971, 'precStd': 0.022912044107604435}


In [7]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  5
Mean Scores (Worst):  {'f1Mean': 0.8251809561464715, 'accMean': 0.8338636284146912, 'recMean': 0.8096930200989059, 'precMean': 0.8632631115330327}
Std Scores (Worst): {'f1Std': 0.015557204667309798, 'accStd': 0.014641758116376876, 'recStd': 0.014712269377250238, 'precStd': 0.018836664816854535}


In [8]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/RF_ustc_argus.json', 'w') as f:
    json.dump(results, f, indent=4)